# 데이터 전처리

In [7]:
import os
import numpy as np
import pandas as pd
import random

data_dir = 'data'

df = pd.read_csv(os.path.join(data_dir, 'ChatbotData.csv'))

In [8]:
# 일상다반사
df[df['label']==0].head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [9]:
# 이별
df[df['label']==1].head()

,Q,A,label
5290,1000일 만난 여자친구와 이별,더 오래 만날 사람 만날 거예요.,1
5291,10년 연애. 헤어졌습니다.,더 공허함이 크시겠네요.,1
5292,10년 연애사 되돌아보니 다 부질없네,더 좋은 사람 만나실 거예요.,1
5293,10년 연예의끝,더 마음이 허하겠어요.,1
5294,10년만나다 헤어지네,충분히 슬퍼하고 충분히 아파하다가 이겨내세요.,1


In [10]:
# 사랑
df[df['label']==2].head()

,Q,A,label
8860,짝사랑만큼 고통스러운 건 없겠지.,짝사랑 만큼 감정소모가 큰 건 없을 거예요.,2
8861,1년 넘게 만났는데 지금도 불타올라,정열적인 사랑을 하고 있나봐요.,2
8862,1년 동거 중인데 계속 좋아,서로 깊게 알게되면서 더 좋아졌나봅니다.,2
8863,1년 동거하고 결혼했어,축하합니다!,2
8864,1년 만났는데도 그 사람에 대해 잘 모르겠어,더 만나보세요.,2


In [11]:
question = df['Q']
answer = df['A']

In [12]:
# 데이터 전처리
import re

# 한글, 영어, 숫자, 공백, ?!.,을 제외한 나머지 문자 제거하는 정규 표현식 패턴 정의
korean_pattern = r'[^ ?,.!A-Za-z0-9가-힣+]'

# 패턴 컴파일
normalizer = re.compile(korean_pattern)
normalizer

re.compile(r'[^ ?,.!A-Za-z0-9가-힣+]', re.UNICODE)

In [13]:
def normalize(sentence):
    return normalizer.sub("", sentence) # sentence 속에서 패턴과 일치하는 부분을 ""로 변경


In [12]:
! pip install konlpy


Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)
   ---------------------------------------- 0.0/351.5 kB ? eta -:--:--
   ------------------- -------------------- 174.1/351.5 kB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 351.5/351.5 kB 5.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---- ----------------------------------- 0.5/3.8 MB 14.5 MB/s eta 0:00:01
   ---------- ----------------------------- 1.0/3.8 MB 13.1 MB/s eta 0:00:01
   ---------------- ----------------------- 1.6/3.8 MB 12.7 MB/s eta 0:00:01
   ---------------------- ----------------- 2.2/3.8 MB 12.5 MB/s eta 0:00:01
   ------------------------ --------------- 2.3/3.8 MB 9.9 MB/s eta 0:00:01
   ----------------------------- ---------- 2.8/3.8 MB 9.8 MB/s eta 0:00:01
   --------------------------------- ------ 3.2/3.8 MB 10.2 MB/s eta 0:00:01
   -------------------------------------- - 3.6/3.8 MB 9.7 MB/s eta 0:00:01
   ---------------------------

In [14]:
# 한글 형태소 분석
from konlpy.tag import Mecab, Okt
okt = Okt()

In [16]:
# 한글 전처리를 함수화
def clean_text(sentence, tagger):
    sentence = normalize(sentence) # 특수문자등 제거
    sentence = tagger.morphs(sentence) # 형태소 분석
    sentence = ' '.join(sentence) # 형태소로 나눠진 sentence list를 ' '으로 연결
    sentence = sentence.lower() # 영어 등 lower
    return sentence

In [17]:
len(question)

11823

In [17]:
# train data 전처리 및 형태소 분석 수행
questions = [clean_text(sent, okt) for sent in question.values[:1000]]
answers = [clean_text(sent, okt) for sent in answer.values[:1000]]

In [19]:
questions[0:3]

['12시 땡 !', '1 지망 학교 떨어졌어', '3 박 4일 놀러 가고 싶다']

In [20]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [22]:
# 단어 사전 WordVocab 생성

PAD_TOKEN = 0
SOS_TOKEN = 1
EOS_TOKEN = 2


class WordVocab():
    def __init__(self):

        # key가 token이고 value가 숫자
        self.word2index = {
            '<PAD>': PAD_TOKEN,
            '<SOS>': SOS_TOKEN, 
            '<EOS>': EOS_TOKEN,
        }

        # 토큰 빈도수 저장
        self.word2count = {}

        # key가 숫자이고 value가 token
        self.index2word = {
            PAD_TOKEN: '<PAD>', 
            SOS_TOKEN: '<SOS>', 
            EOS_TOKEN: '<EOS>'
        }
        
        self.n_words = 3  # PAD, SOS, EOS 포함한 token 개수

    def add_sentence(self, sentence):
        # 문장내 token들에 add_word() 적용
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            # word가 word2index key에 없을 경우, 

            # n_words 번째 숫자 word2index의 key로 추가
            self.word2index[word] = self.n_words

            # word 빈도수 저장 함수에 key(word), value(빈도수) 추가
            self.word2count[word] = 1

            # word2index와 반대 내용 index2word에도 추가
            self.index2word[self.n_words] = word

            # word 개수 + 1
            self.n_words += 1

        # word가 word2index의 key로 있는 경우 word2count에서 해당 word의 value만 +1함    
        else:
            self.word2count[word] += 1

In [23]:
print(f'원문: {question[10]}')
lang = WordVocab()
lang.add_sentence(question[10])
print('==='*10)
print('단어사전')
print(lang.word2index)

원문: SNS보면 나만 빼고 다 행복해보여
단어사전
{'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, 'SNS보면': 3, '나만': 4, '빼고': 5, '다': 6, '행복해보여': 7}


In [25]:
# padding to sequence: 문장 길이 맞추기 코드 예시

max_length = 10 # 패딩된 문장의 최대 길이
sentence_length = 6 # 원본 문장의 길이

# 3~99 사이 정수로 이루어진 size 형태의 랜덤 np 배열 생성 후 list로 변환
sentence_tokens = np.random.randint(low=3, high=100, size=(sentence_length,))
sentence_tokens = sentence_tokens.tolist()
print(f'Generated Sentence: {sentence_tokens}')

# max length보다 길이가 길 경우 9개 크기로 자르고 마지막에 EOS 토큰 추가
sentence_tokens = sentence_tokens[:(max_length-1)]

token_length = len(sentence_tokens)

# 문장의 맨 끝부분에 <EOS> 토큰(2) 추가
sentence_tokens.append(2)

for i in range(token_length, max_length-1):
    # 나머지 빈 곳에 <PAD> 토큰(0) 추가
    sentence_tokens.append(0)

print(f'Output: {sentence_tokens}')
print(f'Total Length: {len(sentence_tokens)}')


Generated Sentence: [50, 34, 25, 66, 48, 41]
Output: [50, 34, 25, 66, 48, 41, 2, 0, 0, 0]
Total Length: 10


## 전처리 프로세스 클래스화

In [ ]:
from konlpy.tag import Mecab, Okt
import os
import pandas as pd
import re 

class TextDataset(Dataset):
    def __init__(self, csv_path, min_length=3, max_length=32):
        super(TextDataset, self).__init__()
        data_dir = 'data'

        # 0,1,2 token 정의
        self.PAD_TOKEN = 0
        self.SOS_TOKEN = 1
        self.EOS_TOKEN = 2

        self.tagger = Okt()
        self.max_length = max_length # 한 문장의 최대 길이

        # 데이터 로드 
        df = pd.read_csv(os.path.join(data_dir, csv_path))

        # 한글 정규화
        korean_pattern = r'[^ ?,.!A-Za-z0-9가-힣+]'
        self.normalizer = re.compile(korean_pattern)

        # src: 질의, tgt: 답변
        src_clean = []
        tgt_clean = []

        # 단어사전 생성
        wordvocab = WordVocab()

        for _, row in df.iterrows():
            src = row['Q']
            tgt = row['A']

            src = self.clean_text(src)
            tgt = self.clean_text(tgt)

            if len(src.split()) > min_length and len(tgt.split()) > min_length:
                # 최소 길이 이상의 문장 단어만 추가
                wordvocab.add_sentence(src)
                wordvocab.add_sentence(tgt)
                src_clean.append(src)
                tgt_clean.append(tgt)
        
        self.srcs = src_clean
        self.tgts = tgt_clean
        self.wordvocb = wordvocab

    def normalize(self, sentence):
        return self.normalizer.sub("", sentence)
    
    def clean_text(self, sentence):
        sentence = self.normalize(sentence)
        sentence = self.tagger.morphs(sentence)
        sentence = ' '.join(sentence)
        sentence = sentence.lower()
        return sentence
    
    def texts_to_sequence(self, sentence):
        # 문장 -> 시퀸스
        return 
